## 17.1.1. Thread-Local Data
Thread-local data就是线程指定的数据。使用`local`或他的子类来创建和存储一个thread -local data

In [1]:
import threading

In [2]:
mydata = threading.local()
mydata.x = 1
mydata.x

1

In [3]:
mydata

## 17.1.2. Thread Objects
```python
class threading.Thread(group=None, target=None, 
                       name=None, args=(),
                       kwargs={}, *, daemon=None)
```
应该使用传关键字调用（如 target=do_something)
**__init__()**
* group: 保留字段；
* target: 传函数。run()时会跑传入的函数，默认None什么都不做；
* args: target传入函数的参数；
* kwargs: target传入函数的关键字参数
* name: 线程名。默认会赋一个唯一的名称，形式为`Thread-N`
* daemon: 如果是None，状态从父线程中继承，如果非None标记为daemon thread

**method：**
* start(): 激活线程, 只能被调用一次
* run(): 执行要线程做的事情。通过target传入的，或复写了
* join(timeout=None): 若thread1在thread2中调用了join()方法thread1.join(), 那么thread2必须等待thread1执行结束、抛出异常或timeout。thread2才继续向下执行。
> 因为join永远return None，所以要调用is_alive()确认time out是否发生。
> 一个线程可以被join()很多次。join()会抛出RuntimeError，如果在尝试join时发生死锁。在线程start()之前join也是一种错误，同样会抛出RuntimeError

* ident: `thread identifier`
* is_alive(): 是否存活。True: 在run()方法运行期间。False: 其他。




如果继承了`Thread`，在开始做其他事情之前，必须回调父类的`__init__()`

`Thread`类控制每一个独立线程的执行动作（指定了这个线程干什么）。
有两种方法指定线程动作：
1. 传递一个可调用对象到构造函数__init__()
2. 在子类中复写run()

这个类只能被重写__init__()和run()

一旦线程对象被创建，激活它必须使用`start()`方法。在控制单独的线程时，`start()`会调用`run()`。

一旦线程被激活，线程被认为是`alive`。其在`run()`方法结束时失去`alive`状态。`run()`方法可能是正常结束，也可能是抛出了一个unhandled exception而结束。可以使用`is_alive()`方法测试线程是否存活(alive)。

其他线程可以调用一个线程的`join()`方法。这会阻塞被调用的线程直到`join()`方法结束。

每一个线程都有一个名字。名字可以传递到构造函数中，通过`name`属性来读或改变

一个线程可以被标记为`daemon thread`。这个标记的意义是只有`daemon thread`去除后Python program才能完全退出。这个的初始值是从父线程中继承来的。这个标记可以通过`daemon`属性设置，或通过构造函数设置。

> Deamon thread是那些在程序关闭时，突然被停止的线程。他们的资源（如已打开的文件句柄，数据库连接等）也许不会被安全的释放。如果你想优雅的停止线程，确认没有制造deamon thread（make them non-daemonic）并且使用一个合适的信号机制（如 Event）

在Python程序中，会有一个主线程对象用于初始化其他线程，这不会是一个`daemon thread`

有时，我们会创建一些`dummy thread`对象。这些线程对象和`alien thread`相关。`alien thread`是控制Python之外的线程模型，如C语言代码创建的线程。`dummy thread`对象有一些功能上的限制：
1. 他们永远被认为是`alive`和`daemonic thread`的
2. 他们不能被join()
3. 因为他们不能被查明，所以他们永远不能被删除

## 17.1.3. Lock Objects
```python
class threading.Lock
```
**method**
* acquire(blocking=True, timeout=-1)
    * blocking: 
        * True, 阻塞直到锁释放，然后将锁锁定并返回True。
        * False，不阻塞。如果一个调用的blocking=True会阻塞，并立即返回False，否则(blockign=False)立即锁定并返回True
    * timeout: 浮点型。正值，无法获取锁时将最多阻塞timeout秒。-1，无线等待。 blocking是False时，timeout会被忽略。
* release(): 释放锁，可以被任何线程调用。如果是一个锁，释放。如果是一组锁，释放一个锁。如果没有锁，raise RuntimeError。没有返回值。


一个primitive lock是一个同步原语。其是一个不属于特定线程的锁。这是Python中最底层的同步原语。

primitive lock有两种状态`locked`,`unlocked`。它在unlocked转态时被撞见。其有两个最基本的方法`acquire()`,`release()`.
* `acquire()`: unlocked ---> locked
* `release()`: locked ---> unlocked
锁支持[上下文管理器协议](https://docs.python.org/release/3.6.8/library/threading.html#with-locks)

## 17.1.4. RLock Objects
重入锁(reentrant lock)是可以被同一个线程多次获取的同步基元组件。其添加了“所属线程”和“递归等级”的概念。在锁定状态下，某些线程拥有锁 ； 在非锁定状态下， 没有线程拥有它。

## 17.1.5. Condition Objects